# Fine-tuning GPT-2
## Write an abstract given the title

## Train

In [1]:
# import libraries

!pip install transformers
from transformers import AutoTokenizer
from transformers import TextDataset,DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments,AutoModelWithLMHead
from transformers import pipeline
import csv
import pandas as pd
import numpy as np

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 74.0 MB/s eta 0:00:00


In [ ]:
# instantiate the tokenizer
tokenizer = AutoTokenizer.from_pretrained('gpt2',sep_token = '<|sep|>')
tokenizer.model_max_length

Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-g

1024

In [ ]:
def load_dataset(train_path,test_path,tokenizer):
    train_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=train_path,
          block_size=128)
     
    val_dataset = TextDataset(
          tokenizer=tokenizer,
          file_path=test_path,
          block_size=128)   
    
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=False,
    )
    return train_dataset,val_dataset,data_collator

# data loading (data already preprocessed by adding special tokens)
train_dataset,val_dataset,data_collator = load_dataset('train_gpt2_abstract.txt','val_gpt2_abstract.txt', tokenizer)

Creating features from dataset file at 
Token indices sequence length is longer than the specified maximum sequence length for this model (661586 > 1024). Running this sequence through the model will result in indexing errors
Saving features into cached file cached_lm_GPT2TokenizerFast_128_train_gpt2_abstract.txt [took 0.009 s]
Creating features from dataset file at 
Saving features into cached file cached_lm_GPT2TokenizerFast_128_val_gpt2_abstract.txt [took 0.003 s]


In [ ]:
# istantiate the model (gpt-2 small with 12-layer, 768-hidden, 12-heads, 117M parameters)
model = AutoModelWithLMHead.from_pretrained("gpt2")

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_vers

In [ ]:
training_args = TrainingArguments(
    output_dir="./model_parameters",  # define the output directory
    overwrite_output_dir=True,        # True if we want to overwrite the content of the output directory
    num_train_epochs=6,               # number of training epochs
    per_device_train_batch_size=16,   # batch size for training
    per_device_eval_batch_size=16,    # batch size for evaluation
    logging_strategy="steps",
    logging_steps=400,
    eval_steps=200,                   # number of update steps between two evaluations
    save_steps=600,                   # number of update steps before saving the update model 
    warmup_steps=200,                 # number of warmup steps for learning rate scheduler
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
import torch

# move the computation on the gpu, if available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cuda


In [ ]:
trainer.train()

***** Running training *****
  Num examples = 5277
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1980
  Number of trainable parameters = 124439808


Step,Training Loss
400,3.907900
800,3.618100
1200,3.485700
1600,3.401600


Saving model checkpoint to ./model_parameters/checkpoint-600
Configuration saved in ./model_parameters/checkpoint-600/config.json
Configuration saved in ./model_parameters/checkpoint-600/generation_config.json
Model weights saved in ./model_parameters/checkpoint-600/pytorch_model.bin
Saving model checkpoint to ./model_parameters/checkpoint-1200
Configuration saved in ./model_parameters/checkpoint-1200/config.json
Configuration saved in ./model_parameters/checkpoint-1200/generation_config.json
Model weights saved in ./model_parameters/checkpoint-1200/pytorch_model.bin
Saving model checkpoint to ./model_parameters/checkpoint-1800
Configuration saved in ./model_parameters/checkpoint-1800/config.json
Configuration saved in ./model_parameters/checkpoint-1800/generation_config.json
Model weights saved in ./model_parameters/checkpoint-1800/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=1980, training_loss=3.552841279000947, metrics={'train_runtime': 1188.4479, 'train_samples_per_second': 26.641, 'train_steps_per_second': 1.666, 'total_flos': 2068257079296000.0, 'train_loss': 3.552841279000947, 'epoch': 6.0})

In [ ]:
# Save
model_path = "./model"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

Configuration saved in ./model/config.json
Configuration saved in ./model/generation_config.json
Model weights saved in ./model/pytorch_model.bin
tokenizer config file saved in ./model/tokenizer_config.json
Special tokens file saved in ./model/special_tokens_map.json


('./model/tokenizer_config.json',
 './model/special_tokens_map.json',
 './model/vocab.json',
 './model/merges.txt',
 './model/added_tokens.json',
 './model/tokenizer.json')

## Generate

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
# Load
model_path = "./model_abstract"
model = AutoModelWithLMHead.from_pretrained(model_path)

In [ ]:
# Generate ans store new data
import pandas as pd
generator = pipeline('text-generation', model=model_path, tokenizer='gpt2', device=0)

def generate_abstract(text):
    # generate the content
    t = generator(text, do_sample=True, max_length=512, top_k=50, top_p=0.50, num_return_sequences=1)[0]['generated_text']
    # remove the input string 
    t = t[len(text):]
    # remove \n and add . if necessary
    t = t.replace('\n', ' ').rsplit('. ', 1)[0] + '.'
    # remove additional parts generated
    if t.find('<|sep|>'):
      x = t.find('<|sep|>')
      t = t[:x].rsplit('. ', 1)[0] + '.'
    return t

titles = pd.read_csv('./gpt2.csv')['title']

# create the output file
with open("./gpt2_from_title_to_abstract.csv",'w') as csvfile:
  # write the header
  writer = csv.DictWriter(csvfile, fieldnames=['title', 'abstract'])
  writer.writeheader()
  # generate the content for each title and append it to the output file
  for elem in titles:
    writer.writerow({"title": elem,'abstract': generate_abstract(str(elem + '<|sep|>'))})

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene